In [ ]:
import pandas as pd
import pathlib
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
experiments = ['openimage', 'google_speech', 'reddit']
EXP_ID = 0
path_data_folder = pathlib.Path(os.path.expandvars(f"$FEDSCALE_HOME/benchmark/logs/monitor/{experiments[EXP_ID]}")).absolute()
print(path_data_folder)
data_filenames = [i for i in sorted(path_data_folder.glob('*'))]
data_filenames

In [ ]:
df0 = pd.read_csv(data_filenames[0])
df0 = df0[(df0[' index'] == 0) | (df0[' index'] == 5)].reset_index().sort_values(by='timestamp', ascending=True)
df0['timestamp'] = pd.to_datetime(df0['timestamp'])
df0['timedelta'] = df0['timestamp'].map(lambda a: a - df0['timestamp'][0]) # since they are synchronized I'm choosing this timestamp as reference
df0['timedelta[s]'] = df0['timedelta'].astype("timedelta64[s]")

df1 = pd.read_csv(data_filenames[1])
df1 = df1[df1[' index'] == 3].reset_index().sort_values(by='timestamp', ascending=True)
df1['timestamp'] = pd.to_datetime(df1['timestamp'])
df1['timedelta'] = df1['timestamp'].map(lambda a: a - df0['timestamp'][0]) # since they are synchronized I'm choosing this timestamp as reference
df1['timedelta[s]'] = df1['timedelta'].astype("timedelta64[s]")

df2 = pd.read_csv(data_filenames[2])
df2 = df2[df2[' index'] == 4].reset_index().sort_values(by='timestamp', ascending=True)
df2['timestamp'] = pd.to_datetime(df2['timestamp'])
df2['timedelta'] = df2['timestamp'].map(lambda a: a - df0['timestamp'][0]) # since they are synchronized I'm choosing this timestamp as reference
df2['timedelta[s]'] = df2['timedelta'].astype("timedelta64[s]")

In [ ]:
df = pd.concat([df0, df1, df2]).reset_index()
df = df.sort_values(by='timestamp', ascending=True)
df

In [ ]:
df[' utilization.gpu [%]'] = df[' utilization.gpu [%]'].astype(str).map(lambda a: a.replace('%', '')).astype(float)
df[' utilization.memory [%]'] = df[' utilization.memory [%]'].astype(str).map(lambda a: a.replace('%', '')).astype(float)
df[' memory.total [MiB]'] = df[' memory.total [MiB]'].astype(str).map(lambda a: a.replace('MiB', '')).astype(float)
df[' memory.free [MiB]'] = df[' memory.free [MiB]'].astype(str).map(lambda a: a.replace('MiB', '')).astype(float)
df[' memory.used [MiB]'] = df[' memory.used [MiB]'].astype(str).map(lambda a: a.replace('MiB', ''))
df = df.dropna()
df[' memory.used [MiB]'] = df[' memory.used [MiB]'].astype(int)
df = df[df['timedelta']<pd.Timedelta("3 minutes")]
df

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(20,10))
sns.lineplot(x='timedelta[s]', y=' utilization.gpu [%]', ax=axs, data=df, hue=' name')
axs.grid()
plt.savefig(os.path.expandvars(f"$FEDSCALE_HOME/{experiments[EXP_ID]}_util_gpu.png"))

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(20,10))
sns.lineplot(x='timedelta[s]', y=' utilization.memory [%]', ax=axs, data=df, hue=' name')
axs.grid()
plt.savefig(os.path.expandvars(f"$FEDSCALE_HOME/{experiments[EXP_ID]}_util_mem.png"))

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(20,10))
sns.lineplot(x='timedelta[s]', y=' memory.used [MiB]', ax=axs, data=df, hue=' name')
axs.grid()
plt.savefig(os.path.expandvars(f"$FEDSCALE_HOME/{experiments[EXP_ID]}_mem_used.png"))